# Personalized Medicine: Redefining Cancer Treatment
- Predict the effect of Genetic Variants to enable Personalized Medicine
- https://www.kaggle.com/c/msk-redefining-cancer-treatment/data

In [16]:
import pandas as pd

# ------------------ Math libraries
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
from scipy.stats import norm
from math import sqrt
from scipy import stats
from scipy import sparse
# ------------------- SKlearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import cross_val_score, train_test_split, learning_curve, validation_curve, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC, LinearSVC
from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_decision_regions, plot_learning_curves, plot_confusion_matrix

# --------------- NLP ----------------
# NLTK
import nltk
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import ToktokTokenizer
# spaCy 
#from spacy.en import English
# gensim
from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
# --------- System ---------
import datetime
import sys
from inspect import getsourcefile
import os.path
import re
import time
import string
import pickle
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
# Input data files are available in the DATA_DIR directory.
DATA_DIR = "data-temp"

# Load data

In [2]:
# Load data. Download from: https://www.kaggle.com/c/msk-redefining-cancer-treatment/data
train_text = pd.read_csv(DATA_DIR + "/training_text.csv", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
train_class = pd.read_csv(DATA_DIR + "/training_variants.csv")
test_text = pd.read_csv(DATA_DIR + "/test_text.csv", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
test_class = pd.read_csv(DATA_DIR + "/test_variants.csv")

In [ ]:
print("Train text:", train_text.shape, " train class:", train_class.shape)
print("Test text:", test_text.shape, " test class:", test_class.shape)

In [ ]:
train_text[:2]

In [ ]:
train_class[:2]

In [3]:
# combine train_text + train_class
train_full = train_class.merge(train_text, how="inner", left_on="ID", right_on="ID")
train_full.head(5)

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [4]:
# combine test_text + test_class
test_full = test_class.merge(test_text, how="inner", left_on="ID", right_on="ID")
test_full.head(5)

,ID,Gene,Variation,Text
0,0,ACSL4,R570S,2. This mutation resulted in a myeloproliferat...
1,1,NAGLU,P521L,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,PAH,L333F,Vascular endothelial growth factor receptor (V...
3,3,ING1,A148D,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,TMEM216,G77A,Abstract Retinoblastoma is a pediatric retina...


In [5]:
label = "Class"
# Initial features
o_features = test_full.columns.values
# Store eval_id
eval_data_id = test_full['ID'].values
#Seperate input and label from train_data
input_data = train_full[o_features]
target = train_full[label]
target.describe()


count    3321.000000
mean        4.365854
std         2.309781
min         1.000000
25%         2.000000
50%         4.000000
75%         7.000000
max         9.000000
Name: Class, dtype: float64

In [6]:
#Combine train + eval data
combine_data = pd.concat([input_data, test_full], keys=['train','eval'])
data = combine_data
data.head(5)

ID    Gene             Variation  \
train 0   0  FAM58A  Truncating Mutations   
      1   1     CBL                 W802*   
      2   2     CBL                 Q249E   
      3   3     CBL                 N454D   
      4   4     CBL                 L399V   

                                                      Text  
train 0  Cyclin-dependent kinases (CDKs) regulate a var...  
      1   Abstract Background  Non-small cell lung canc...  
      2   Abstract Background  Non-small cell lung canc...  
      3  Recent evidence has demonstrated that acquired...  
      4  Oncogenic mutations in the monomeric Casitas B...

# Explore data

### Check Gene categories

In [ ]:
col = 'Gene'
print(data[col].head(5))
data[col].value_counts().describe()

### Check Variants categories

In [ ]:
col = 'Variation'
print(data[col].head(5))
data[col].value_counts().describe()

In [7]:
# Transform target
# Label encode the targets
labels = LabelEncoder()
target_tf = labels.fit_transform(target)
target = target_tf
target

array([0, 1, 1, ..., 0, 3, 3])

### Check NaN columns

In [ ]:
def check_null_data(data):
    #Get high percent of NaN data
    null_data = data.isnull()
    total = null_data.sum().sort_values(ascending=False)
    percent = (null_data.sum()/null_data.count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    high_percent_miss_data = missing_data[missing_data['Percent']>0]
    #print(missing_data)
    print(high_percent_miss_data)
    miss_data_cols = high_percent_miss_data.index.values
    return miss_data_cols

In [ ]:
check_null_data(data)

# Modeling

### Tokenize function based on NLTK

In [ ]:
# Credit: http://nlpforhackers.io/recipe-text-clustering/
stemmer = PorterStemmer()
def tokenize_nltk(text):
    toktok = ToktokTokenizer()
    #tokens =[toktok.tokenize(sent) for sent in sent_tokenize(text)]
    tokens = nltk.word_tokenize(text)
    stems = [stemmer.stem(t) for t in tokens]
    #print("Number of tokens:", len(tokens))
    return stems

### Tokenize function based on spaCy

In [ ]:
# credit: https://nicschrading.com/project/Intro-to-NLP-with-spaCy/
parser = English()
# A custom stoplist
STOPLIST = set(sw.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve"]
# Every step in a pipeline needs to be a "transformer". 
# Define a custom transformer to clean text using spaCy
class CleanTextTransformer(TransformerMixin):
    """
    Convert text to cleaned text
    """

    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}
    
    # A custom function to clean the text before sending it into the vectorizer
    def cleanText(self, text):
        # get rid of newlines
        text = text.strip().replace("\n", " ").replace("\r", " ")

        # replace twitter @mentions
        mentionFinder = re.compile(r"@[a-z0-9_]{1,15}", re.IGNORECASE)
        text = mentionFinder.sub("@MENTION", text)

        # replace HTML symbols
        text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")

        # lowercase
        text = text.lower()

        return text

# A custom function to tokenize the text using spaCy
# and convert to lemmas
def tokenize_spacy(sample):

    # get the tokens using spaCy
    tokens = parser(sample)

    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]

    # stoplist symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLS]

    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

## Word2Vec tranform

In [8]:
# credit: https://github.com/nadbordrozd/blog_stuff/blob/master/classification_w2v/benchmarking.ipynb
def word2vec(data, max_features=2000, min_df=5, n_jobs=4, load=False):
    # train word2vec on all the texts - both training and test set
    # we're not using test labels, just texts so this is fine
    model_file = DATA_DIR + "/word2vec"
    if load:
        model = Word2Vec.load(model_file)
    else:    
        start = time.time()
        model = Word2Vec(data, size=max_features, min_count=min_df, workers=n_jobs)
        end = time.time() - start
        model.save(model_file)
        print("Word2vec training time:", end)
    w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}
    return w2v

In [23]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values())
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        array = np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ]) 
        return sparse.csr_matrix(array) 
    def fit_transform(self, X):
        self.fit(X, None)
        return self.transform(X)

### Trainng word2vec model for combine_data

In [10]:
data = combine_data['Text'].values
start = time.time()
w2v = word2vec(data, load=True)
end = time.time() - start
end

0.020415306091308594

In [ ]:
len(w2v)

## Transform features

In [28]:
def text_features_w2v(data, load=False):
    filename = DATA_DIR + "/tfidf.npz"
    if load:
        with open(filename, 'rb') as infile:
            array = pickle.load(infile)
    else:
        w2v = word2vec(data, load=True)
        tfidf = TfidfEmbeddingVectorizer(w2v)
        start = time.time()
        array = tfidf.fit_transform(data)
        end = time.time() - start
        with open(filename, 'wb') as outfile:
            pickle.dump(array, outfile, pickle.HIGHEST_PROTOCOL)
        print("Text feature Transform time:", end)
    return array    

In [ ]:
#tfidf = TfidfVectorizer(min_df=5, max_features=2000, stop_words=sw.words('english'), tokenizer=tokenize, lowercase=True)
#time = 27.434333562850952
#vectorize = CountVectorizer(min_df=5, max_features=2000, stop_words=sw.words('english'), tokenizer=tokenize, lowercase=True)
#0.7735180854797363
#vectorizer = CountVectorizer(min_df=5, max_features=2000, stop_words=sw.words('english'), lowercase=True)
# 27.707584381103516
#vectorizer = HashingVectorizer(n_features=2000, stop_words=sw.words('english'), tokenizer=tokenize, lowercase=True)
# 0.9716482162475586
#vectorize = HashingVectorizer(n_features=2000, stop_words=sw.words('english'), lowercase=True)
# 49.8653244972229 (100 records)
vectorizer = CountVectorizer(min_df=5, max_features=2000, tokenizer=tokenize_spacy)

In [ ]:
# 2946 sec
#tfidf = TfidfVectorizer(min_df=5, max_features=2000, stop_words=sw.words('english'), tokenizer=tokenize_nltk, lowercase=True)
# time = 78.87074208259583
#tfidf = TfidfVectorizer(min_df=5, max_features=2000, stop_words=sw.words('english'), lowercase=True)
# time =49.89134097099304 sec (100 records)
#tfidf = TfidfVectorizer(min_df=5, max_features=2000, tokenizer=tokenize_spacy, lowercase=True)

In [ ]:
# time = 68.3006434440612 (100 records)
tfidf = TfidfEmbeddingVectorizer(w2v)

### Test transformation

In [29]:
data1 = train_full['Text'][:100]
start = time.time()
#tf = vectorizer.fit_transform(data1)
#tf = tfidf.fit_transform(data1)
tf = text_features_w2v(data1, load=False)
end = time.time() - start
end

0.13696599006652832

In [30]:
tf1 = tf[:50]
tf2 = tf[5:]
print("tf size:", tf.shape)
print("tf1 size:", tf1.shape)
#tf11 = tf1.todense()
#tf11[:5]

tf size: (100, 2000)
tf1 size: (50, 2000)


In [ ]:
tfidf1 = TfidfTransformer(use_idf=False)

In [ ]:
data = tf
#data = tfidf1.fit_transform(tf)
y = target[:data.shape[0]]
X_train, X_test, Y_train, Y_test = train_test_split(data, y, train_size=0.7, random_state=324)
model1 = LinearSVC()
model1.fit(X_train, Y_train)
model1.score(X_test, Y_test)

### Tranform combine_data

In [31]:
# time = 78.87074208259583 => tokenize = None
# time = 2946.212862968445 => tokenize = word_tokenize
# time = 5492.0656995773315 => tokenize = spacy
data = combine_data['Text'].values
start = time.time()
#data_tf = tfidf.fit_transform(data)
data_tf = text_features_w2v(data, load=False)
end = time.time() - start
end

Text feature Transform time: 27.413230419158936


27.7857027053833

### Split train set and eval set

In [ ]:
train_len = len(train_full)
train_set = data_tf[:train_len]
eval_set = data_tf[train_len:]
print("Train set:", train_set.shape, " eval set:", eval_set.shape)

### Model pipe line

In [ ]:
# Score: 0.2, submission score: 1.64639
#model = SVC(decision_function_shape='ovo', probability=True, random_state=250)

In [ ]:
model = XGBClassifier(n_estimators=500, max_depth=5, n_jobs = -1)

In [ ]:
#preprocessor = NLTKPreprocessor()

In [ ]:
model_pipeline = Pipeline([('tfidf', tfidf),
                            ('model', model),
                 ])


### Split train set in to train and test set

In [ ]:
data = train_set
print(data.shape)
data[:5]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data, target, train_size=0.7, random_state=324)
print("train size:", X_train.shape)
print("test size:", X_test.shape)

### Fit model

In [ ]:
# time: 298.4094178676605
start = time.time()
model.fit(X_train, Y_train)
end = time.time() - start
end

### Evaluate model

In [ ]:
start = time.time()
score = model.score(X_test, Y_test)
end = time.time() - start
print("time:", end, " score:", score)

### Confusion matrix

In [ ]:
start = time.time()
y_pred = model.predict(X_test)
end = time.time() - start
end

In [ ]:
cm = confusion_matrix(y_target=Y_test, 
                      y_predicted=y_pred)
plot_confusion_matrix(conf_mat=cm)

# Predict eval set and export

In [ ]:
#predictions
# time = 3.4443037509918213
data_eval = eval_set
start = time.time()
y_pred=model.predict_proba(data_eval)
end = time.time() - start
end

In [ ]:
y_pred[:10]

In [ ]:
#tweaking the submission file as required
# credit: https://www.kaggle.com/punyaswaroop12/gbm-starter-top-40
subm_file = pd.DataFrame(y_pred)
subm_file['id'] = eval_data_id
subm_file.columns = ['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8', 'class9', 'id']
subm_file.to_csv(DATA_DIR+ "/submission_v2.csv",index=False)
subm_file.head(5)

In [ ]:
subm_file.shape